In [1]:
import Rwire_to_accuracy_lib as rlf
import torch
import torchvision
import time
import torch.nn.functional as F
import torch.nn as nn

In [2]:
#载入数据集
dataset_train = torchvision.datasets.MNIST(root='./../all_dataset',train=True,transform=torchvision.transforms.ToTensor(),download=False)
dataset_test = torchvision.datasets.MNIST(root='./../all_dataset',train=False,transform=torchvision.transforms.ToTensor(),download=False)

dataloader_train = torch.utils.data.DataLoader(dataset=dataset_train,batch_size=100,shuffle=True)
dataloader_test = torch.utils.data.DataLoader(dataset=dataset_test,batch_size=1,shuffle=False)


In [10]:
# Relux B=1 1.1258  B=2 1.1057  B=3 1.1032
# tanhx B=1 0.8205  B=2 0.6473  B=3 0.6084  B=5 1.3194
# sigmoid B=2.0346 1.9337  B=2 1.4480  B=3 1.0811  B=5 0.9589


B=7
w_factor = 1.9337
PATH = './Lenetq8_sigmoidx.pt'
model_param = torch.load(PATH)

for param in model_param:
    print(param)
    print(model_param[param].shape)
    print(model_param[param].abs().max())
    
c1_weight = model_param['conv1.weight']
c1_bias = model_param['conv1.bias']
c2_weight = model_param['conv2.weight']
c2_bias = model_param['conv2.bias']
fc1_weight = model_param['fc1.weight']
fc1_bias = model_param['fc1.bias']
fc2_weight = model_param['fc2.weight']
fc2_bias = model_param['fc2.bias']
fc3_weight = model_param['fc3.weight']
fc3_bias = model_param['fc3.bias']

#xbar定义
Rm_min, Rm_max, Rm_off = 10e3, 50e3, 10e9
xbar_row, xbar_column = 128,128
R_row, R_column = 3,3

conv1.weight
torch.Size([6, 1, 5, 5])
tensor(0.6378)
conv1.bias
torch.Size([6])
tensor(0.8583)
conv2.weight
torch.Size([16, 6, 5, 5])
tensor(0.3622)
conv2.bias
torch.Size([16])
tensor(0.0945)
fc1.weight
torch.Size([120, 400])
tensor(0.2126)
fc1.bias
torch.Size([120])
tensor(0.0630)
fc2.weight
torch.Size([84, 120])
tensor(0.3622)
fc2.bias
torch.Size([84])
tensor(0.0866)
fc3.weight
torch.Size([10, 84])
tensor(1.)
fc3.bias
torch.Size([10])
tensor(0.4882)


In [4]:
st1 = time.time()
G_c1p, G_c1n=rlf.W_to_G(c1_weight, c1_bias, Rm_min, Rm_max, Rm_off, xbar_row, xbar_column, linear=False)
G_c1p = G_c1p.cuda()
G_c1n = G_c1n.cuda()
A_c1p_inv = rlf.matrix_Ainv_generate_big_cuda(G_c1p,R_row,R_column)
A_c1n_inv = rlf.matrix_Ainv_generate_big_cuda(G_c1n,R_row,R_column)
print(G_c1p.shape,G_c1n.shape,A_c1p_inv.shape,A_c1n_inv.shape)
ed1 = time.time()
print(ed1-st1)

st2 = time.time()
G_c2p, G_c2n=rlf.W_to_G(c2_weight, c2_bias, Rm_min, Rm_max, Rm_off, xbar_row, xbar_column, linear=False)
G_c2p = G_c2p.cuda()
G_c2n = G_c2n.cuda()
A_c2p_inv = rlf.matrix_Ainv_generate_big_cuda(G_c2p,R_row,R_column)
A_c2n_inv = rlf.matrix_Ainv_generate_big_cuda(G_c2n,R_row,R_column)
print(G_c2p.shape,G_c2n.shape,A_c2p_inv.shape,A_c2n_inv.shape)
ed2 = time.time()
print(ed2-st2)

st3 = time.time()
G_fc1p, G_fc1n=rlf.W_to_G(fc1_weight, fc1_bias, Rm_min, Rm_max, Rm_off, xbar_row, xbar_column, linear=True)
G_fc1p = G_fc1p.cuda()
G_fc1n = G_fc1n.cuda()
A_fc1p_inv = rlf.matrix_Ainv_generate_big_cuda(G_fc1p,R_row,R_column)
A_fc1n_inv = rlf.matrix_Ainv_generate_big_cuda(G_fc1n,R_row,R_column)
print(G_fc1p.shape,G_fc1n.shape,A_fc1p_inv.shape,A_fc1n_inv.shape)
ed3 = time.time()
print(ed3-st3)

st4 = time.time()
G_fc2p, G_fc2n=rlf.W_to_G(fc2_weight, fc2_bias, Rm_min, Rm_max, Rm_off, xbar_row, xbar_column, linear=True)
G_fc2p = G_fc2p.cuda()
G_fc2n = G_fc2n.cuda()
A_fc2p_inv = rlf.matrix_Ainv_generate_big_cuda(G_fc2p,R_row,R_column)
A_fc2n_inv = rlf.matrix_Ainv_generate_big_cuda(G_fc2n,R_row,R_column)
print(G_fc2p.shape,G_fc2n.shape,A_fc2p_inv.shape,A_fc2n_inv.shape)
ed4 = time.time()
print(ed4-st4)

st5 = time.time()
G_fc3p, G_fc3n=rlf.W_to_G(fc3_weight, fc3_bias, Rm_min, Rm_max, Rm_off, xbar_row, xbar_column, linear=True)
G_fc3p = G_fc3p.cuda()
G_fc3n = G_fc3n.cuda()
A_fc3p_inv = rlf.matrix_Ainv_generate_big_cuda(G_fc3p,R_row,R_column)
A_fc3n_inv = rlf.matrix_Ainv_generate_big_cuda(G_fc3n,R_row,R_column)
print(G_fc3p.shape,G_fc3n.shape,A_fc3p_inv.shape,A_fc3n_inv.shape)
ed5 = time.time()
print(ed5-st5)

torch.Size([1, 1, 128, 128]) torch.Size([1, 1, 128, 128]) torch.Size([1, 1, 33024, 128]) torch.Size([1, 1, 33024, 128])
27.197622776031494
torch.Size([2, 1, 128, 128]) torch.Size([2, 1, 128, 128]) torch.Size([2, 1, 33024, 128]) torch.Size([2, 1, 33024, 128])
42.21311020851135
torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 128, 128]) torch.Size([4, 1, 33024, 128]) torch.Size([4, 1, 33024, 128])
84.59133672714233
torch.Size([1, 1, 128, 128]) torch.Size([1, 1, 128, 128]) torch.Size([1, 1, 33024, 128]) torch.Size([1, 1, 33024, 128])
20.754520893096924
torch.Size([1, 1, 128, 128]) torch.Size([1, 1, 128, 128]) torch.Size([1, 1, 33024, 128]) torch.Size([1, 1, 33024, 128])
22.520469903945923


In [11]:
vsource = 0.3
vin_level = 2**8-1  #8bit  对应正负输入电压范围
h_level = 2**7-1
delta_v = vsource/vin_level
delta_g = (1/Rm_min-1/Rm_max)/h_level
delta_i = delta_v*delta_g
w_scale = 1/(1/Rm_min-1/Rm_max)

error_idx = []

fmc1_set_after = []
fmc2_set_after = []
fmfc1_set_after = []
fmfc2_set_after = []

correct = 0
st = time.time()
for batch_idx,(fm,label) in enumerate(dataloader_test):
    fm = fm.cuda()
    label = label.cuda()
    if batch_idx%100 == 99:
        print(batch_idx)
        print('correct=',correct)
    # print(batch_idx)
    # print('label=',label)
    z_set_c1 = rlf.vector_z_generate_conv_cuda((fm[0]),xbar_row,xbar_column,kernel_h=5,kernel_w=5,stride_h=1,stride_w=1,padding = 2,bias = True)
    z_set_c1 = (z_set_c1*vin_level).round()*delta_v
    Iout_c1p = rlf.conv_forward_cuda(A_c1p_inv,z_set_c1,G_c1p.shape,r_column=R_column,Cout=6)
    Iout_c1n = rlf.conv_forward_cuda(A_c1n_inv,z_set_c1,G_c1n.shape,r_column=R_column,Cout=6)
    Iout_c1 = (Iout_c1p-Iout_c1n)/vsource*w_scale*w_factor
    Iout_c1 = F.max_pool2d(torch.sigmoid(B*Iout_c1),2)


    
    z_set_c2= rlf.vector_z_generate_conv_cuda(Iout_c1,xbar_row,xbar_column,kernel_h=5,kernel_w=5,stride_h=1,stride_w=1,padding = 0,bias = True)
    z_set_c2 = (z_set_c2*vin_level).round()*delta_v
    Iout_c2p = rlf.conv_forward_cuda(A_c2p_inv,z_set_c2,G_c2p.shape,r_column=R_column,Cout=16)
    Iout_c2n = rlf.conv_forward_cuda(A_c2n_inv,z_set_c2,G_c2n.shape,r_column=R_column,Cout=16)
    Iout_c2 = (Iout_c2p-Iout_c2n)/vsource*w_scale*w_factor
    Iout_c2 = F.max_pool2d(torch.sigmoid(B*Iout_c2),2).reshape(-1)


    
    z_set_fc1 = rlf.vector_z_generate_linear_cuda(Iout_c2,xbar_row,xbar_column,bias = True)
    z_set_fc1 = (z_set_fc1*vin_level).round()*delta_v
    Iout_fc1p = rlf.linear_forward_cuda(A_fc1p_inv,z_set_fc1,G_fc1p.shape,R_column,out_neuron_num=120)
    Iout_fc1n = rlf.linear_forward_cuda(A_fc1n_inv,z_set_fc1,G_fc1n.shape,R_column,out_neuron_num=120)
    Iout_fc1 = (Iout_fc1p-Iout_fc1n)/vsource*w_scale*w_factor
    Iout_fc1 = torch.sigmoid(B*Iout_fc1)


  
    z_set_fc2 = rlf.vector_z_generate_linear_cuda(Iout_fc1,xbar_row,xbar_column,bias = True)
    z_set_fc2 = (z_set_fc2*vin_level).round()*delta_v
    Iout_fc2p = rlf.linear_forward_cuda(A_fc2p_inv,z_set_fc2,G_fc2p.shape,R_column,out_neuron_num=84)
    Iout_fc2n = rlf.linear_forward_cuda(A_fc2n_inv,z_set_fc2,G_fc2n.shape,R_column,out_neuron_num=84)
    Iout_fc2 = (Iout_fc2p-Iout_fc2n)/vsource*w_scale*w_factor
    Iout_fc2 = torch.sigmoid(B*Iout_fc2)


    
    z_set_fc3 = rlf.vector_z_generate_linear_cuda(Iout_fc2,xbar_row,xbar_column,bias = True)
    z_set_fc3 = (z_set_fc3*vin_level).round()*delta_v
    Iout_fc3p = rlf.linear_forward_cuda(A_fc3p_inv,z_set_fc3,G_fc3p.shape,R_column,out_neuron_num=10)
    Iout_fc3n = rlf.linear_forward_cuda(A_fc3n_inv,z_set_fc3,G_fc3n.shape,R_column,out_neuron_num=10)
    Iout_fc3 = (Iout_fc3p-Iout_fc3n).reshape(1,10)

    value,idx = torch.max(Iout_fc3,1)
    # print(idx)
    correct += (idx==label).sum()
ed = time.time()
print(ed-st)
print('correct=',correct)

99
correct= tensor(98, device='cuda:0')
199
correct= tensor(194, device='cuda:0')
299
correct= tensor(288, device='cuda:0')
399
correct= tensor(383, device='cuda:0')
499
correct= tensor(480, device='cuda:0')
599
correct= tensor(577, device='cuda:0')
699
correct= tensor(674, device='cuda:0')
799
correct= tensor(770, device='cuda:0')
899
correct= tensor(867, device='cuda:0')
999
correct= tensor(962, device='cuda:0')
1099
correct= tensor(1056, device='cuda:0')
1199
correct= tensor(1150, device='cuda:0')
1299
correct= tensor(1240, device='cuda:0')
1399
correct= tensor(1332, device='cuda:0')
1499
correct= tensor(1425, device='cuda:0')
1599
correct= tensor(1519, device='cuda:0')
1699
correct= tensor(1612, device='cuda:0')
1799
correct= tensor(1706, device='cuda:0')
1899
correct= tensor(1804, device='cuda:0')
1999
correct= tensor(1899, device='cuda:0')
2099
correct= tensor(1989, device='cuda:0')
2199
correct= tensor(2079, device='cuda:0')
2299
correct= tensor(2174, device='cuda:0')
2399
corre

In [4]:
#模型定义
class Lenet5(nn.Module):
    def __init__(self):
        super(Lenet5,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(6,16,5,stride=1,padding=0)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = F.max_pool2d(F.sigmoid(self.conv1(x*w_factor)),2)
        x = F.max_pool2d(F.sigmoid(self.conv2(x*w_factor)),2)
        x = x.view(x.shape[0],-1)   ###x.size to (batchsize,-1])
        x = F.sigmoid(self.fc1(x*w_factor))
        x = F.sigmoid(self.fc2(x*w_factor))
        x = self.fc3(x)
        return x
    
#####识别率验证############    
model1 = Lenet5()
model_param1 = torch.load(PATH)
model1.load_state_dict(model_param1)

correct = 0    
for j, (images, labels) in enumerate(dataloader_test):
    pred = model1(images)
    values,indices = torch.max(pred,1)
    correct += (indices==labels).sum()
print(correct)

/root/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(9655)
